# Projekt 2 - Hőmérséklet mentése fájlba

Mint házi ügyeletes szakit, a szüleid megbíztak, hogy készíts egy szoba hőmérsékletét rögzítő berendezést, ugyanis szeretnék látni és kielemezni, hogy különböző időpontokban, hogy változik az otthonotokban a hőmérséklet. 

Arra kértek, hogy készíts egy programot, ami az általad megadott ideig méri, az általad megadott időközönként az otthon hőmérsékletét. Az eredményt pedig egy fájlba menti le, ami tartalmazza a mérés időpontját és a hőmérsékletet Celsius és Fahrenheit fokokban.

## Mit fogsz készíteni?

Egy DS18b20 szenzorból (és ha kell egy ellenállásból) álló áramkört készítünk, amiből másodpercenként kiolvassuk a hőmérsékletet 30 másodpercen keresztül. Mind a kiolvasás időpontját és a hőmérsékleteket elmentjük egy fájlba, hogy később is felkutathassuk a mért értékeket.  

## Mit tanulsz meg?

A hőmérő szenzor elkészítésével a következőket tanulod meg:

* Hogyan futtas terminál parancsokat Pythonból.
* Hogyan keress fájlokat és mappákat az operációs rendszeredben Pythonból a ```glob``` csomaggal.
* Hogyan olvassuk ki egy szöveges fájl tartalmát az ```open``` parancs használatával.
* Hogyan nyerjünk ki hasznos információt egy szöveges fájlból string parancsokat használva.
* Hogyan kezeljük a dátum formátumokat a ```datetime``` csomag segítségével.
* Hogyan mentsünk el adatokat fájlokba az ```open``` segítségével.

## A projekt részletekre bontása

* Elkészíteni az áramkört.
* Előkészíteni a kommunikációt a thermisztorral (```modprobe``` parancsok futtatása).
* Definiálni a hőmérsékletet tartalmazó fájl lokációját.
* Kiolvasni a fájl tartalmát az ```open``` parancs segítségével.
* Definiálni egy függvényt ami a fájlból kinyeri a hőmérsékletet és a kiolvasás időpontját.
* Definiálni egy függvényt, ami az időpontot és a hőmérsékletet olyan formába alakítja, hogy kimenthető legyen fájlba.
* Definiálni egy függvényt ami kimenti az adatokat egy fájlba.
* Másodpercenként megismételni a hőmérséklet olvasást.
* Ellenőrizni, hogy eltelt-e a mérésre megadott időtartam.

## Áramköri elemek listája

a) [Raspberry PI](https://malnapc.hu/yis/raspberry-pi/rpi-panelek) 

b) DS18b20 szenzor: [itt vásárolhatsz](https://www.tme.eu/hu/details/ds18b20+/homerseklet-jelatalakitok/maxim-integrated/)

c) [Jumper wires female/male](https://www.ret.hu/shop/product/e-call/jumper-vezetek-szet_53-22-63) 

d) Ellenállás: [itt vásárolhatsz](https://www.tme.eu/hu/katalog/metal-film-tht-ellenallasok_112313/?s_order=desc&search=ellenallas&s_field=1000011)

## A kapcsolási rajz

<img src="programok/prog01_schema.png" width=600 height=400 />

A fenti ábrához hasonlóan kapcsoljuk össze az áramköri elemeket és a Raspberry Pi-t. Az áramkör részletes összekötési magyarázata a [Thermisztorok]() című bevezető leírásban található.

1) Kössük össze a Raspberry Pi egyik földelését a szenzor GND jelű lábával (fekete drót).

2) A szenzor Vcc és Signal lábait kössük össze egy 4.7 kOhmos ellenállással (ez a lépés nem szükséges, ha ez az ellenállás már alapból be van építve). 

3) A szenzor Signal lábát (középső láb) kössük össze a Raspberryn a *GPIO04*-es tüskével (kék drót). 

4) A szenzor Vcc lábát kössük a Raspberry 3.3 V-os táp tüskéjére (piros drót). 

## A kód

Nyissunk meg egy új python fájlt és mentsük el pl. ```ds18b20_save2file.py``` név alatt. A ```gpiozero``` csomagnak nincs beépített objektuma ami általánosan a thermisztorokkat tudná kezelni, így minden egyes thermisztorhoz nekünk kell megoldani a szoftveres kommunikációt az adott eszközzel.

### A DS18b20 hőmérséklet fájljának előkészítése

Miután elkészítettük az áramkört, meg kell írnunk a kódot ami utasítja a Raspberry Pi-t, a hőmérséklet kiolvasására.

Mint tudjuk a hőmérséklet értékeket a Raspberry egy kernel parancs elindításával fogja majd a *w1_slave* nevű fájlban tárolni. Az előző projektben már megtanultuk, hogyan olvashatjuk ki a hőmérsékletet a fájl tartalmából és most azt fogjuk újra hasznosítani egy kis módosítással. 

Most a hőmérséklet kiolvasásának pillanatát is rögzítenünk kell és ehhez a ```datetime``` csomagot fogjuk használni.

#### A datetime csomag

A ```datetime``` csomag klasszokat tartalmaz a dátumok és időpontok létrehozására és az azok közti műveletek elvégzésére. Első lépésben beimportáljuk:

In [1]:
import datetime as dt

Nézzük, hogy jeleníthetjük meg a mostani pillanatot dátummal együtt, illetve milyen műveleteket lehet rajta elvégezni:

In [12]:
now = dt.datetime.today()         # elmentjuk a mostani pillanatot, 
                                  # EV-HONAP-NAP ORA:PERC:MS
print(now)
tomorrow = dt.datetime(2021,2,17) # letrehozunk egy datetime valtozot
print(tomorrow)
print(now.minute)                 # lekerjuk a valtozo PERC erteket
print(type(now))                  # kinyomtatjuk a datetime tipusat
now_string = now.strftime('%d/%B/%Y') # atalakitjuk a datetime objektumot 
                                      # stringge NAP/HONAP/EV alakban
print(now_string)
print(type(now_string))                       
date_str = '1999-08-11 15:12:33'               # string formaban idopont definialas
# string idoforma datetime formava alakitasa
date_dt = dt.datetime.strptime(date_str, '%Y-%m-%d %H:%M:%S') 
print(date_dt)
print(date_dt.month)
print(type(date_dt))

2021-02-16 17:33:13.800384
2021-02-17 00:00:00
33
<class 'datetime.datetime'>
16/February/2021
<class 'str'>
1999-08-11 15:12:33
8
<class 'datetime.datetime'>


Számunkra jelenleg a legfontosabb dolog amit észben kell tartani, hogy:

* miként tudjuk megállapítani a jelen pillanatot, ```datetime.datetiem.today()``` vagy ```datetime.datetiem.now()```, 
* miként tudjuk a ```datetime``` klasszt string formátummá alakítani, hiszen stringként fogjuk kiírni a fájlba, ```datetime.datetime.strftime()```.

#### A hőmérséklet kiolvasási időpontjának rögzítése

Hasznosítsuk újra az előző projekt kódjának számunkra megfelelő részét, másoljuk ki az új fájlunkba. Ezt egészítsük ki a következő sorokkal (a lenti kódban a kommentelt részek jelzik a hozzáadott sorokat):

* importáljuk be a ```datetime``` csomagot
* a ```read_temp(file_name)``` függvényben olvassuk ki a hőmérsékletet, így itt rögzítjük a kiolvasás pillanatát a ```now = dt.datetime.today().strftime('%Y-%m-%d %H:%M:%S')``` sorban. A ```'%Y-%m-%d %H:%M:%S'``` kifejezéssel adjuk meg, hogy milyen formátumban szeretnénk a fájlunkban megjeleníteni az időpontot, Év-Hónap-Nap Óra-Perc-Másodperc,
* a függvény végén a hőmérsékletekkel együtt a mérés időpontját is visszaadjuk,
* egy végtelen ciklusban kiolvassuk a hőmérsékletet és időpillanatot, kinyomtatjuk és másodpercenként megismételjük a műveletet.

```ds18b20_save2file.py```:

In [1]:
import os
import glob
import time
import datetime as dt                 # importáljuk a datetime csomagot
 
os.system('modprobe w1-gpio')
os.system('modprobe w1-therm')
 
base_dir = '/sys/bus/w1/devices/'
device_folder = glob.glob(base_dir + '28*')[0]
device_file = device_folder + '/w1_slave'
 
def read_temp_raw(file_name):
    f = open(file_name, 'r')
    lines = f.readlines()
    f.close()
    return lines
 
def read_temp(file_name):
    lines = read_temp_raw(file_name)
    # rögzítjuk az aktuális időpontot
    now = dt.datetime.today().strftime('%Y-%m-%d %H:%M:%S')  
    while lines[0].strip()[-3:] != 'YES':
        time.sleep(0.2)
        lines = read_temp_raw(file_name)
    equals_pos = lines[1].find('t=')
    if equals_pos != -1:
        temp_string = lines[1][equals_pos+2:]
        temp_c = float(temp_string) / 1000.0
        temp_f = temp_c * 9.0 / 5.0 + 32.0
        return temp_c, temp_f, now      # visszaadjuk a hőmérsékleteket és az időpontot
    
while True:                             # indítunk egy végtelen ciklust
	tc, tf, ts = read_temp(device_file) # kiolvassuk a hőmérsékletet egy adott pillanatban
    print(ts, tc, tf)                   # kinyomtatjuk a fenti értékeket
    time.sleep(1)                       # várunk egy másodpercet a következő mérés előtt

### Adatok előkészítése mentéshez

Létrehozunk egy függvényt, ami a *csv* (comma separated values) fájlformátumnak megfelelően rendszerezi az adatainkat. Ez csak annyit tesz, hogy az egymáshoz tartozó értékeket egy sorba írjuk és vesszővel választjuk el őket. A függvényünk, ```prepare2save(tc, tf, ts)```, három bemenő paramétert fogad el, a Celsius és Fahrenheit hőmérsékleteket, valamint a mérés időpontját. A sorok első eleme a mérés időpontja lesz, a második a Celsius hőmérséklet, a harmadik pedig a Fahrenheit hőmérséklet.

Szöveges fájlba kimenteni adatokat csak akkor lehet, ha az adat típusa string. Így a fent kinyert hőmérsékletet és időpillanatot elő kell készítenünk a kimentésre. Az időpillanat már string formátumban van, így avval nem kell sokat törődni, de a két számot már át kell alakítani string formátumba. 

Első lépésként a hőmérsékleteket lekerekítjük kettő tizedes pontosságra, ```round(tc,2)```, majd átalakítjuk string formátumba a ```str(round(tc,2))``` paranccsal. Pythonban stringeket simán az ```+``` jellel tudjuk bővíteni, így a menteni kívánt sorunk a következőképpen fog kinézni:```txt = ts + ',' + str(round(tc,2)) + ',' + str(round(tf,2)) + '\n'```. Fontos, hogy a sor végére odategyük az új sor jelét, ```\n```, különben egy sorba mentjük ki az összes mérést. 

In [13]:
def prepare2save(tc,tf,ts):
	txt = ts + ',' + str(round(tc,2)) + ',' + str(round(tf,2)) + '\n'
	return txt

Nézzünk egy példát, hogy jeleníti majd meg a bemenő adatokat:

In [15]:
print(prepare2save(24.456, 74.231, '1999-08-11 15:12:33'))

1999-08-11 15:12:33,24.46,74.23



### Függvény az adatok kimentéséhez

A következő lépésben definiálunk egy függvényt, ```write2file(txt,filename='temperature.csv')``` aminek bemenő paramétereként az elmentendő stringet adjuk meg és hogy mi legyen a fájl neve amibe mentjük. 

In [16]:
def write2file(txt,filename='temperature.csv'):
	f = open(filename, 'a')
	f.write(txt)
	f.close()

Mivel, a hőmérséklet mérések nem egyszerre történnek, hanem minden egyes ciklusban újabbat mérünk, ezért a fájlt az ```'a'``` append vagyis kiegészít argumentummal látjuk el. Így nem íródik fölül a fájl tartalma, hanem minden egyes ciklusban bővül. 

### A 30 másodperces hőmérséklet mérés fájlba mentése

Nem maradt más hátra, mint a fent leírt programrészleteket egybegyúrni és kiegészíteni a hőmérséklet mérés hosszának rögzítésével. Ehhez még a kód elején definiálunk két dolgot: a fájl nevét, ```filename```, amibe el szeretnénk menteni az adatokat illetve, hogy milyen hosszan szeretnénk mérni, ```duration```. A lenti kódban kommentelt sorok jelzik az új információkat a kódból. 

A függvények definiálása után, kezdőthet a fő rész. Időméréshez két változót definiálunk, ```t0```: jelzi a mérés kezdetét, ```t1```: jelzi az épp aktuális mérési ciklus pillanatát. A kettő különbsége adja meg, hogy mennyi idő telt el a mérés kezdete óta. Ha ez a különbség nagyobb mint a ```duration```-ben definiált időtartam, akkor kilépünk a ```while``` ciklusból és véget ér a python futása. 

Ha viszont az eltelt idő kisebb, mint a ```duration```, akkor:

* kiolvassuk a hőmérsékleteket, 
* rögzítjük az aktuális ciklus pillanatát, ```t1```, 
* kinyomtatjuk a képernyőre a mérés kezdetétől eltelt időt másodpercben,
* előkészítjük mentésre az adatokat, ```prepare2save```,
* elmentjük az adatokat az előre megadott fájlnév alá, ```write2file```,
* várunk egy másodpercet a következő mérésig.

```ds18b20_save2file.py```:

In [ ]:
import os
import glob
import time
import datetime as dt
 
os.system('modprobe w1-gpio')
os.system('modprobe w1-therm')
 
base_dir = '/sys/bus/w1/devices/'
device_folder = glob.glob(base_dir + '28*')[0]
device_file = device_folder + '/w1_slave'

filename = 'test.csv'                           # a fájl neve amibe mentünk
duration = 30                                   # a mérés hossza
 
def read_temp_raw(file_name):
    f = open(file_name, 'r')
    lines = f.readlines()
    f.close()
    return lines
 
def read_temp(file_name):
    lines = read_temp_raw(file_name)
    now = dt.datetime.today().strftime('%Y-%m-%d %H:%M:%S')
    while lines[0].strip()[-3:] != 'YES':
        time.sleep(0.2)
        lines = read_temp_raw(file_name)
    equals_pos = lines[1].find('t=')
    if equals_pos != -1:
        temp_string = lines[1][equals_pos+2:]
        temp_c = float(temp_string) / 1000.0
        temp_f = temp_c * 9.0 / 5.0 + 32.0
        return temp_c, temp_f, now

def write2file(txt,filename='temperature.csv'):
	f = open(filename, 'a')
	f.write(txt)
	f.close()
	
def prepare2save(tc,tf,ts):
	txt = ts + ',' + str(round(tc,2)) + ',' + str(round(tf,2)) + '\n'
	return txt
	
t0 = time.time()                       # mérés kezdetének pillanata
t1 = time.time()                       # az aktuális ciklus kezdetének pillanata

while t1 - t0 < duration:              # eltelt-e 30 másodperc a mérés kezdete óta
	tc, tf, ts = read_temp(device_file)       
	t1 = time.time()                   # az aktuális ciklus időpillanata
	print(t1-t0)                       # a mérés kezdetétől eltelt idő megjelenítése
	text = prepare2save(tc, tf, ts)    # a mért adatok kimentésre előkészítése
	write2file(text, filename=filename)# mért adatok kimentése
	time.sleep(1)

## A projekt tesztelése

Miután összeszereltük az áramkört és a kódot is megírtuk, amit pl. ```ds18b20_save2file.py``` név alatt mentettünk el, megnyithatunk a Raspberry Pi operációs rendszerén egy terminált. A terminálban a ```cd 'mappa név'``` paranccsal elnavigálunk abba a mappába, ahova a ```ds18b20_save2file.py```-t elmentettük. Ott begépelve a ```python ds18b20_save2file.py``` parancsot, letesztelhetjük a programunk működését. Ha minden jól megy akkor a program elindításával a elmentjük a 30 másodpercen keresztül mért adatokat a *test.csv* nevű fájlba. Közben a képernyőn megjelenik, hogy mennyi idő telt el a kód elindítása óta és a számláló 30 másodpercig megy.

Hibaüzenetek esetén ki kell deríteni mi lehetett a probléma, pl. elgépelés, egy modul hiányzik, sorok megfelelő behúzása, idézőjel lemaradása stb. A hibaüzenet legtöbbször segít abban, hogy melyik sorban találta a hibát és hogy mi volt az. Egy kis gyakorlással bele lehet jönni azok értelmezésébe, valamint interneten is rá lehet keresni a hibaüzenet jelentésére és annak lehetséges elhárítására.

## Mit lehet javítani/továbbfejleszteni?

* Változtassuk a mérés hosszát és gyakoriságát.
* Módosítsuk a mért adatok lementését úgy, hogy az első sor a fejléc legyen, és a következőket tartalmazza: Datum,t_C,t_F.

Írd meg kommentben, hogy szerinted mivel lehetne még feldobni ezt a kis programot!

## Referencia

1) https://thepihut.com/blogs/raspberry-pi-tutorials/18095732-sensors-temperature-with-the-1-wire-interface-and-the-ds18b20

2) http://docs.37sensors.com/#

3) https://sensorkit.en.joy-it.net/index.php?title=KY-028_Temperature_Sensor_module_(Thermistor)

4) https://www.malnasuli.hu/leckek/homerseklet-merese-1-wire-szenzorral/

5) https://www.electronicshub.org/raspberry-pi-ds18b20-tutorial/

6) open parancs - https://www.guru99.com/reading-and-writing-files-in-python.html

7) glob csomag - https://pymotw.com/2/glob/

8) os csomag - https://www.python-course.eu/os_module_shell.php

9) string metódusok - https://www.programiz.com/python-programming/string

10) datetime csomag - https://www.programiz.com/python-programming/datetime